### Haciendo uso de gen_smart_negatives
Para mejorar la calidad del entrenamiento, usamos gen_smart_negatives, que genera ejemplos negativos más realistas. En lugar de seleccionar productos aleatorios, esta estrategia elige productos similares (misma categoría) que el cliente no compró. Esto permite entrenar modelos más robustos y simula decisiones reales de compra, evitando que el modelo aprenda patrones triviales.





In [14]:
import pandas as pd
import numpy as np

# Cargar positivos
df_pos = pd.read_csv("customer_purchases_train.csv")
df_pos['label'] = 1

# Función de negativos inteligentes
def gen_smart_negatives(df, n_per_positive=2, seed=42):
    np.random.seed(seed)
    cat_cliente = df.groupby('customer_id')['item_category'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
    positive_pairs = set(zip(df['customer_id'], df['item_id']))
    cat_to_items = df.groupby('item_category')['item_id'].unique().to_dict()
    negatives = []

    for customer in df['customer_id'].unique():
        cat_pref = cat_cliente.get(customer)
        if cat_pref is None or cat_pref not in cat_to_items:
            continue
        candidate_items = cat_to_items[cat_pref]
        purchased_items = df[df['customer_id'] == customer]['item_id'].unique()
        non_purchased = np.setdiff1d(candidate_items, purchased_items)
        sampled_items = np.random.choice(non_purchased, size=min(n_per_positive, len(non_purchased)), replace=False)
        for item in sampled_items:
            negatives.append({'customer_id': customer, 'item_id': item, 'label': 0})

    return pd.DataFrame(negatives)

# Generar negativos
df_neg = gen_smart_negatives(df_pos, n_per_positive=2)

# Unir positivos y negativos
train_df_full = pd.concat([df_pos, df_neg], ignore_index=True)

# Guardar CSV para entrenamiento
train_df_full.to_csv("train_df_full.csv", index=False)
print("train_df_full.csv guardado con negativos inteligentes")

# Cargar perfiles
customer_profile = pd.read_csv("customer_profile.csv")  # ya lo tienes
product_profile = pd.read_csv("product_profile.csv")

# Enriquecer train_df_full con atributos del cliente y producto
train_df_full = train_df_full.merge(customer_profile, on="customer_id", how="left")
train_df_full = train_df_full.merge(product_profile, on="item_id", how="left")

# Guardar CSV enriquecido
train_df_full.to_csv("train_df_full_enriched.csv", index=False)
print("train_df_full_enriched.csv guardado con atributos completos")



train_df_full.csv guardado con negativos inteligentes
train_df_full_enriched.csv guardado con atributos completos


### Verificacion de negativos y balance


In [15]:
df = pd.read_csv("train_df_full_enriched.csv")
print(df.isnull().sum())
print(df["label"].value_counts())
print(df.head())


purchase_id               1000
customer_id                  0
customer_date_of_birth    1000
customer_gender_x         2551
customer_signup_date      1000
item_id                      0
item_title_x              1000
item_category_x           1000
item_price_x              1000
item_img_filename         1000
item_avg_rating_x         1045
item_num_ratings_x        1000
item_release_date_x       1000
purchase_timestamp        1000
customer_item_views       1000
purchase_item_rating      6745
purchase_device           1000
label                        0
customer_gender_y            0
edad                         0
total_compras                0
gasto_promedio               0
antiguedad_dias              0
categoria_frecuente          0
blouse                       0
dress                        0
jacket                       0
jeans                        0
shirt                        0
shoes                        0
skirt                        0
slacks                       0
suit    

### CSV para enriquecer la generacion de negativos 

In [7]:
# Crear perfil por producto
product_profile = df_pos.drop_duplicates("item_id")[[
    "item_id", "item_category", "item_price", "item_title",
    "item_avg_rating", "item_num_ratings", "item_release_date"
]]

# Guardar CSV
product_profile.to_csv("product_profile.csv", index=False)
print("✅ product_profile.csv guardado")


✅ product_profile.csv guardado
